# Transfer Learning in Keras

In this notebook, we'll cover how to load a pre-trained model (in this case, VGGNet19) and finetune it for a new task: Oxford Flower.

#### Load dependencies

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.applications.vgg19 import VGG19


# from keras.applications.vgg19 import VGG19
# from keras.models import Sequential
# from keras.layers import Dense, Dropout, Flatten
# from keras.preprocessing.image import ImageDataGenerator

In [ ]:
#!pip install tflearn

import tflearn.datasets.oxflower17 as oxflower17
X, Y = oxflower17.load_data(one_hot=True)

#### Load the pre-trained VGG19 model

In [ ]:
vgg19 = VGG19(include_top=False,
              weights='imagenet',
              input_shape=(224,224,3),
              pooling=None)

#### Freeze all the layers in the base VGGNet19 model

In [ ]:
for layer in vgg19.layers:
    layer.trainable = False

#### Add custom classification layers

In [ ]:
# Instantiate the sequential model and add the VGG19 model: 
model = Sequential()
model.add(vgg19)

# Add the custom layers atop the VGG19 model: 
model.add(Flatten(name='flattened'))
model.add(Dropout(0.5, name='dropout'))
model.add(Dense(17, activation='softmax', name='predictions'))

#### Compile the model for training

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(X, Y, batch_size=64, epochs=10, verbose=1, validation_split=0.1, shuffle=True)

#### Using Image Generator

https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator

https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator#flow

In [ ]:
from sklearn.model_selection import train_test_split

#Split train and test data
X_train, X_valid, y_train, y_valid = train_test_split(X, Y, test_size=0.2,shuffle = True)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Instantiate two image generator classes:
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    data_format='channels_last',
    rotation_range=30,
    horizontal_flip=True,
    fill_mode='reflect')

valid_datagen = ImageDataGenerator(
    rescale=1.0/255,
    data_format='channels_last')

In [ ]:
# Define the batch size:
batch_size=64

In [ ]:
# Define the train and validation generators: 
train_generator = train_datagen.flow(X_train, y_train, batch_size=batch_size, shuffle=True, seed=42)

valid_generator = valid_datagen.flow(X_valid, y_valid, batch_size=batch_size, shuffle=True, seed=42)

In [ ]:
model.fit(train_generator, steps_per_epoch=15, epochs=10, validation_data=valid_generator, validation_steps=15, verbose=1)